In [ ]:
!pip install googlemaps python-dotenv

In [3]:
import dotenv
import os
import json
from googlemaps import Client
from pprint import pprint

In [22]:
dotenv.load_dotenv()

gmaps = Client(key=os.getenv("GMAPS_API_KEY"))


SEARCH_PARAMS = {
    "radius": 8000,
    "type": "restaurant",
    "language": 'en'
}
MAX_RESULT_PAGES = 50
RAW_DATASETS_PATH = 'data/raw'
CSV_DATASETS_PATH = 'data/csv'

if not os.path.exists(RAW_DATASETS_PATH):
    os.makedirs(RAW_DATASETS_PATH)

if not os.path.exists(os.path.join(RAW_DATASETS_PATH, 'reviews')):
    os.makedirs(os.path.join(RAW_DATASETS_PATH, 'reviews'))    

if not os.path.exists(CSV_DATASETS_PATH):
    os.makedirs(CSV_DATASETS_PATH)    

if not os.path.exists(os.path.join(CSV_DATASETS_PATH, 'reviews')):
    os.makedirs(os.path.join(CSV_DATASETS_PATH, 'reviews'))    

In [5]:
def save_raw_results(results, idx: int):
    name_segs = {k:v for k,v in sorted(SEARCH_PARAMS.items(), key=lambda item: item[0], reverse=True)}
    
    name = '_'.join(map(str,name_segs.values())) + "-" + str(idx) +'.json'
    with open(os.path.join(RAW_DATASETS_PATH, name), 'w') as f:
        json.dump(results, f)


## Fetch RAW results

In [44]:
# Get the latitude and longitude of Padova
padova = gmaps.geocode("Padova")[0]["geometry"]["location"]

# Get all the places in Padova
results_0 = gmaps.places(location=padova, **SEARCH_PARAMS)
save_raw_results(results_0, 0)

In [6]:
idx = 1
while results_0.get("next_page_token", None) and idx < MAX_RESULT_PAGES:
    page_token = results_0["next_page_token"]
    results = gmaps.places(page_token=page_token)
    results['page_token'] = page_token

    save_raw_results(results, idx)

    idx += 1


NameError: name 'results_0' is not defined

## Read RAW results

In [7]:
def get_all_results():
    for p in os.scandir(RAW_DATASETS_PATH):
        if p.name.startswith('review'): continue

        with open(os.path.join(RAW_DATASETS_PATH, p.name)) as f:
            try:
                yield from json.load(f)['results']
            except Exception as e:
                pass


In [10]:
all_results = list(get_all_results())
print("Num of all results:", len(all_results))

Num of all results: 1000


In [27]:
def is_reviews_exist(reviews):
    name = 'reviews-' + reviews['result']['place_id'] + '.json'
    return os.path.exists(os.path.join(RAW_DATASETS_PATH, 'reviews', name))

In [28]:
def save_raw_reviews(reviews):
    name = 'reviews-' + reviews['result']['place_id'] + '.json'
    with open(os.path.join(RAW_DATASETS_PATH, 'reviews', name), 'w') as f:
        json.dump(reviews, f)

## Fetch RAW reviews

In [30]:
all_reviews = []
for res in all_results:
    # print("res:",res)
    # print("res['place_id']:", res['place_id'])
    # restaurant = gmaps.place(res['place_id'])
    # print("restaurant:", restaurant)
    # print("reviews['result']['place_id']:", restaurant['result']['place_id'])
    # print('is_reviews_exist(restaurant):', is_reviews_exist(restaurant))
    # break
    print('fetching place id:', res['place_id'], end='')
    restaurant = gmaps.place(res['place_id'])
    all_reviews.append(restaurant)
    
    print(' ... done => ', end='')
    if is_reviews_exist(restaurant): 
        print('ALREADY EXISTS')
        continue
    save_raw_reviews(restaurant)
    print('SAVED')


fetching place id: ChIJPTRpx3jbfkcRx5Gv5a-P484 ... done => ALREADY EXISTS
fetching place id: ChIJeeU78IjZfkcReoht8Oi1dQs ... done => ALREADY EXISTS
fetching place id: ChIJn0ipN6fbfkcRANpZ-fvlgl0 ... done => ALREADY EXISTS
fetching place id: ChIJpThkkazbfkcR_ZAfff-TNO4 ... done => ALREADY EXISTS
fetching place id: ChIJsyFieFfafkcRN_O0j_MdLPk ... done => ALREADY EXISTS
fetching place id: ChIJpW5wYu_QfkcRn9XltKPMj1w ... done => ALREADY EXISTS
fetching place id: ChIJLUDgA1DafkcR-dtY0yPNtyA ... done => ALREADY EXISTS
fetching place id: ChIJ42w6oiTFfkcR29FnNZ0Qu3w ... done => ALREADY EXISTS
fetching place id: ChIJL7_PE1fafkcRgSh4yy4s7hY ... done => ALREADY EXISTS
fetching place id: ChIJM40mU7LQfkcRPGT8E6UbtrI ... done => ALREADY EXISTS
fetching place id: ChIJAQtKXRbXfkcR7Rqeuk3FXhc ... done => ALREADY EXISTS
fetching place id: ChIJMStIPpTefkcR6vzlMHX-Kuc ... done => ALREADY EXISTS
fetching place id: ChIJP39wIffafkcR_HAseeSiBZ4 ... done => ALREADY EXISTS
fetching place id: ChIJVRdlLHfbfkcR0It

KeyboardInterrupt: 

In [23]:
pprint(restaurant)

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '741',
                                    'short_name': '741',
                                    'types': ['street_number']},
                                   {'long_name': 'Riva del Vin',
                                    'short_name': 'Riva del Vin',
                                    'types': ['route']},
                                   {'long_name': 'Venezia',
                                    'short_name': 'Venezia',
                                    'types': ['locality', 'political']},
                                   {'long_name': 'Venezia',
                                    'short_name': 'Venezia',
                                    'types': ['administrative_area_level_3',
                                              'political']},
                                   {'long_name': 'Città Metropolitana di '
                                                 'Venezia',
                    

In [28]:
list(map(lambda rest:rest['results'],results))

TypeError: string indices must be integers, not 'str'